In [326]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stat
import numpy.linalg as LA
import pysal
from pysal.esda.moran import Moran
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

In [357]:
#Compute Eigenvalues and Eigenvectors from the 
#transformed Spatial weights Matrix usuibg python/numpy

#It says in the paper that a rook-case binary contiguity wieghts matrix
#is being used, so that is what I employ here.
neighbor_list = '/users/toshan/projects/GWR_SF/georgia_rook.gal'
style = 'b'


w = pysal.open(neighbor_list).read()
w.transform = style
C, ids = pysal.full(w)
#C = 0.5 * (C + C.T)
n = C.shape[0]
M = np.identity(n) - (np.float(1)/np.float(n))
MCM = np.dot(np.dot(M,C),M)
vals, vecs = LA.eig(MCM)

pairs = zip(vals, vecs)
svals = sorted(vals, reverse=True)


stups = [None] * len(svals)
for e_val, vector in pairs:
    stups[svals.index(e_val)] = (e_val, vector)

In [358]:
new_vecs = [x[1] for x in stups]
new_vals = [x[0] for x in stups]

In [359]:
#Compute Eigenvalues and Eigenvectors from the 
#transformed Spatial weights Matrix using R

#It says in the paper that a rook-case binary contiguity wieghts matrix
#is being used, so that is what I employ here.
spdep = importr("spdep")
georgia_rook = spdep.read_gal("/Users/toshan/projects/GWR_SF/georgia_rook.gal", override_id = True)
base = importr("base")

M_r = np.array(ro.r('diag(1,159)-1/159'))
C_r = spdep.nb2listw(georgia_rook, style="B")
C_r = np.array(spdep.listw2mat(C_r))

ro.globalenv["M_r"] = M_r
ro.globalenv["C_r"] = C_r

MCM_r = ro.r('M_r%*%C_r%*%M_r')
ro.globalenv["MCM_r"] = MCM_r

vals_r, vecs_r = MCM_r.eigen()
vals_r, vecs_r = np.array(vals_r), np.array(vecs_r)

In [378]:
#Correlations for sorted Python eigenvectors and R eigenvectors
s_cors = []
for i, vec in enumerate(new_vecs):
    s_cors.append(stat.pearsonr(vec, vecs_r[i])[0])
s_cors = np.array(s_cors)


In [379]:
#Correlations for unsorted Python eigenvectors and R eigenvectors
cors = []
for i, vec in enumerate(vecs):
    cors.append(stat.pearsonr(vec, vecs_r[i])[0])
cors = np.array(cors)


In [380]:
print np.allclose(new_vals, vals_r)
print np.allclose(vals, vals_r)
print np.allclose(vals, new_vals)
print np.allclose(cors, s_cors)



True
False
False
False


In [383]:
cors[cors > .25]
s_cors[s_cors > .25]

array([], dtype=float64)

In [356]:
#Same results for Spatial Weight
print np.array_equal(C_r, C)

#Same results for M
print np.array_equal(M_r, M)

#Results for MCM vary slightly between R and Pthyon - seems like rounding errors
print np.array_equal(MCM_r, MCM)

#Rounding to the 14th digit results in consistent MCM between R and Python
print np.array_equal(np.round(MCM_r, decimals=14), np.round(MCM, decimals=14))

                     
#Same results for Eigenvalues if we similarly round them after sorting them
print np.array_equal(np.sort(np.round(vals,11)), np.sort(np.round(vals_r,11)))

#Same does not hold for Eigenvectors, which led me to the fact that the python and R eigenvectors
#were provided in differnt orders
print np.array_equal(np.sort(np.round(vecs,4)), np.sort(np.round(vecs_r,4)))

True
True
False
True
True
False


In [178]:
#Great, We've got some Eigenvectors! Lets try plugging them into
#the formulas provided in the paper.

#First we need to extract the top 40 eigenvectors with the highest eigenvalues
#We'll add them to a pandas DF for ease.

os.chdir('/users/toshan/projects/GWR_SF/')
georgia = pd.read_csv('georgia.csv')

for x, eig in enumerate(vecs[:40]):
    georgia['E' + str(x+1)] = eig

intercept = 11.65947
elderly = .01827 - .57455*georgia['E7'] + 2.68529*georgia['E9'] - 1.43008*georgia['E32']
pop = .0004740 + .00053444*georgia['E5'] - .0034627*georgia['E21'] + .00027500*georgia['E28'] + .00008588*georgia['E29'] - .00049085*georgia['E37'] + .0052024*georgia['E38']
rural = -.02164 - .40283*georgia['E9'] - .07260*georgia['E17'] - .29333*georgia['E23'] - .21945*georgia['E30'] + .17920*georgia['E37']
foreign = 2.53632 + 7.88189*georgia['E1'] + 14.51402*georgia['E2'] - 4.80336*georgia['E4'] - 10.02322*georgia['E5'] - 9.54772*georgia['E9'] + 6.13165*georgia['E13'] + 4.46841*georgia['E15'] - 8.26015*georgia['E20'] + 8.95200*georgia['E22'] + 3.31062*georgia['E25'] - 8.83627*georgia['E28'] - 6.88889*georgia['E32'] - 17.36410*georgia['E38'] + 5.23183*georgia['E40']
poverty = -.26300 + .80033*georgia['E23'] + .26817*georgia['E28'] + .54815*georgia['E30']
black = .06104 - .7218*georgia['E1'] + .13764*georgia['E21'] - .17630*georgia['E22'] + .65026*georgia['E32']

georgia['yhat'] = intercept + elderly*georgia['PctEld'] + pop*georgia['TotPop90'] + rural*georgia['PctRural'] + foreign*georgia['PctFB'] + poverty*georgia['PctPov'] + black*georgia['PctBlack']
print stat.pearsonr(georgia['PctBach'], georgia['yhat'])[0]**2

0.313731663628


In [179]:
#Not very close to the reported fit of 0.93
#But wait, lets see if our eigenvalues are close to theirs.
#If we looked at the vals we'd see that they're not scaled to the
#general Moran's domain (-1,1) since we used a binary weight. The paper
#actually reports Moran's coefficients instead of actualy eigenvalues so
#lets take a look at the top 40 eigenvector's MC's

w = pysal.open(neighbor_list).read()
MI = []
for each in vecs[:40]:
    m = Moran(each, w, transformation='b')
    MI.append(m.I)
    
MI.sort(reverse=True)
MI

[0.10448653361131389,
 0.076140779515824611,
 0.072720103656055343,
 0.066742103014807236,
 0.062247974045844347,
 0.058050034998503523,
 0.057977498647291632,
 0.045549148999462982,
 0.037783834296204792,
 0.037624002443017224,
 0.03640423981342597,
 0.028449480400275538,
 0.025053904776756328,
 0.018336882539394746,
 0.013977439715316259,
 0.013031579949651113,
 0.010503431093578939,
 0.0077354210591788035,
 -0.00075475127515737606,
 -0.00090198026256855981,
 -0.0019608260931500438,
 -0.0031493798712722855,
 -0.0051250815817740023,
 -0.0067279709707271981,
 -0.011566697762258073,
 -0.016639608601696439,
 -0.018480415430264446,
 -0.019313043753337724,
 -0.02452887472720491,
 -0.026149476224149454,
 -0.02805140350859053,
 -0.040993113916661592,
 -0.049049818585298013,
 -0.05545582343064924,
 -0.058172131148083756,
 -0.060466032597548798,
 -0.066596352997525421,
 -0.082326781208239511,
 -0.085567147878208183,
 -0.090293544448943996]

In [180]:
#Well thats strange, there seems to be no SA, when by definition there should be.
#Lets try peeling them off the opposite dimension by using vals.T instead of vals

w = pysal.open(neighbor_list).read()
MI = []
for each in vecs.T[:40]:
    m = Moran(each, w, transformation='b')
    MI.append(m.I)
    
MI.sort(reverse=True)
MI

[1.0790660847227049,
 1.0418805220804448,
 1.0254977042841851,
 0.98288559452996738,
 0.94308034331807766,
 0.92444784132219193,
 0.90495775740328233,
 0.85669143119083457,
 0.85115772662466915,
 0.83438328129510331,
 0.79659733031871272,
 0.77749974310264747,
 0.76292173407078001,
 0.73623347723171972,
 0.70575011581586544,
 0.67969050969027489,
 0.65225770070344014,
 0.64421442458043721,
 0.62934596196578274,
 0.60447710019149792,
 0.5841577446836167,
 0.57732643351044288,
 0.54222417708678006,
 0.52866738622362475,
 0.50942277926472168,
 0.49567322553247867,
 0.46708533059505353,
 0.45097228173196219,
 0.43662916903997762,
 0.42817432335901945,
 0.3996026601284049,
 0.37967473255956191,
 0.36164779405152925,
 0.35274354867300139,
 0.34757301695302945,
 0.33140953215011926,
 0.32290682938587134,
 0.31160329742398585,
 0.30711081186478345,
 0.27733499873276879]

In [181]:
#These eigenvalues are very close to those presented in the paper,
#except they are consistently slightly lower. Let's see how well we 
#can reproduce their formulas using vecs.T instead of vecs.

for x, eig in enumerate(vecs.T[:40]):
    georgia['E' + str(x+1)] = eig

intercept = 11.65947
elderly = .01827 - .57455*georgia['E7'] + 2.68529*georgia['E9'] - 1.43008*georgia['E32']
pop = .0004740 + .00053444*georgia['E5'] - .0034627*georgia['E21'] + .00027500*georgia['E28'] + .00008588*georgia['E29'] - .00049085*georgia['E37'] + .0052024*georgia['E38']
rural = -.02164 - .40283*georgia['E9'] - .07260*georgia['E17'] - .29333*georgia['E23'] - .21945*georgia['E30'] + .17920*georgia['E37']
foreign = 2.53632 + 7.88189*georgia['E1'] + 14.51402*georgia['E2'] - 4.80336*georgia['E4'] - 10.02322*georgia['E5'] - 9.54772*georgia['E9'] + 6.13165*georgia['E13'] + 4.46841*georgia['E15'] - 8.26015*georgia['E20'] + 8.95200*georgia['E22'] + 3.31062*georgia['E25'] - 8.83627*georgia['E28'] - 6.88889*georgia['E32'] - 17.36410*georgia['E38'] + 5.23183*georgia['E40']
poverty = -.26300 + .80033*georgia['E23'] + .26817*georgia['E28'] + .54815*georgia['E30']
black = .06104 - .7218*georgia['E1'] + .13764*georgia['E21'] - .17630*georgia['E22'] + .65026*georgia['E32']

georgia['yhat'] = intercept + elderly*georgia['PctEld'] + pop*georgia['TotPop90'] + rural*georgia['PctRural'] + foreign*georgia['PctFB'] + poverty*georgia['PctPov'] + black*georgia['PctBlack']
print stat.pearsonr(georgia['PctBach'], georgia['yhat'])[0]**2

0.460927034654


In [182]:
#Ok slightly better but still nowhere near theirs. There is the issue of the 
#non-sorted eigenvectors/vals so lets try this again with the results from R 
#that seem to be sorted.

for x, eig in enumerate(vecs_r[:40]):
    georgia['E' + str(x+1)] = eig

intercept = 11.65947
elderly = .01827 - .57455*georgia['E7'] + 2.68529*georgia['E9'] - 1.43008*georgia['E32']
pop = .0004740 + .00053444*georgia['E5'] - .0034627*georgia['E21'] + .00027500*georgia['E28'] + .00008588*georgia['E29'] - .00049085*georgia['E37'] + .0052024*georgia['E38']
rural = -.02164 - .40283*georgia['E9'] - .07260*georgia['E17'] - .29333*georgia['E23'] - .21945*georgia['E30'] + .17920*georgia['E37']
foreign = 2.53632 + 7.88189*georgia['E1'] + 14.51402*georgia['E2'] - 4.80336*georgia['E4'] - 10.02322*georgia['E5'] - 9.54772*georgia['E9'] + 6.13165*georgia['E13'] + 4.46841*georgia['E15'] - 8.26015*georgia['E20'] + 8.95200*georgia['E22'] + 3.31062*georgia['E25'] - 8.83627*georgia['E28'] - 6.88889*georgia['E32'] - 17.36410*georgia['E38'] + 5.23183*georgia['E40']
poverty = -.26300 + .80033*georgia['E23'] + .26817*georgia['E28'] + .54815*georgia['E30']
black = .06104 - .7218*georgia['E1'] + .13764*georgia['E21'] - .17630*georgia['E22'] + .65026*georgia['E32']

georgia['yhat'] = intercept + elderly*georgia['PctEld'] + pop*georgia['TotPop90'] + rural*georgia['PctRural'] + foreign*georgia['PctFB'] + poverty*georgia['PctPov'] + black*georgia['PctBlack']
print stat.pearsonr(georgia['PctBach'], georgia['yhat'])[0]**2

0.145357636666


In [183]:
# And for the other dimension of vecs.

for x, eig in enumerate(vecs_r.T[:40]):
    georgia['E' + str(x+1)] = eig

intercept = 11.65947
elderly = .01827 - .57455*georgia['E7'] + 2.68529*georgia['E9'] - 1.43008*georgia['E32']
pop = .0004740 + .00053444*georgia['E5'] - .0034627*georgia['E21'] + .00027500*georgia['E28'] + .00008588*georgia['E29'] - .00049085*georgia['E37'] + .0052024*georgia['E38']
rural = -.02164 - .40283*georgia['E9'] - .07260*georgia['E17'] - .29333*georgia['E23'] - .21945*georgia['E30'] + .17920*georgia['E37']
foreign = 2.53632 + 7.88189*georgia['E1'] + 14.51402*georgia['E2'] - 4.80336*georgia['E4'] - 10.02322*georgia['E5'] - 9.54772*georgia['E9'] + 6.13165*georgia['E13'] + 4.46841*georgia['E15'] - 8.26015*georgia['E20'] + 8.95200*georgia['E22'] + 3.31062*georgia['E25'] - 8.83627*georgia['E28'] - 6.88889*georgia['E32'] - 17.36410*georgia['E38'] + 5.23183*georgia['E40']
poverty = -.26300 + .80033*georgia['E23'] + .26817*georgia['E28'] + .54815*georgia['E30']
black = .06104 - .7218*georgia['E1'] + .13764*georgia['E21'] - .17630*georgia['E22'] + .65026*georgia['E32']

georgia['yhat'] = intercept + elderly*georgia['PctEld'] + pop*georgia['TotPop90'] + rural*georgia['PctRural'] + foreign*georgia['PctFB'] + poverty*georgia['PctPov'] + black*georgia['PctBlack']
print stat.pearsonr(georgia['PctBach'], georgia['yhat'])[0]**2

0.359500877476


In [184]:
#Better than the previous dimension, but lower than the potentially unsorted python results.

#False alarm on the incorrect dimension argument. But The fact the 
#Pythyon unsorted results perform better than the sorted R results could be
#problematic. 

#Must find a way to match my Eigenvectors/vals/MC's to the ones reported in the paper. 
#Otherwise validation will be useless. 

